In [17]:
import pandas
import pandas as pd
import numpy as np
import sys
import sklearn
from sklearn.metrics import log_loss, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
from tqdm import tqdm

deepctr_path = '/root/linghui/rec-fairness/'
sys.path.append(deepctr_path)
import deepctr
from deepctr.models import DeepFM
from deepctr.feature_column import SparseFeat, VarLenSparseFeat, get_feature_names
# from utilis import get_feat_dict

In [28]:
def get_feat_dict(data):
    if data == 'ml-1m':
        feat_dict = {'movie_id': 3706,
                 'user_id': 6040,
                 'gender': 2,
                 'age': 7,
                 'occupation': 21,
                 'zipcode': 3439}
    if data == 'black-fri':
        feat_dict = {'user_id': 5891,
                 'product_id': 3631,
                 'gender': 2,
                 'age': 7,
                 'occupation': 21,
                 'city': 3,
                 'stay_years': 5,
                 'martial_status': 2,
                 'cate_1': 20,
                 'cate_2': 18,
                 'cate_3': 16,
                 'price': 23962}
    return feat_dict

In [5]:
#load data

saved_model_path = '/data/linghui/saved_model/deepfm-black-fri/'

saved_data_path = '/data/linghui/black-friday/processed_data/'
# train_data_path = saved_data_path + 'train_data.csv' 
# test_data_path = saved_data_path + 'test_data.csv'
train_data_path = saved_data_path + 'train_data_liuyi.csv' 
test_data_path = saved_data_path + 'test_data_liuyi.csv'


#load data
train_data = pd.read_csv(train_data_path,index_col=0)

/opt/conda/lib/python3.8/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [6]:
train_data

,user_id,product_id,gender,age,occupation,city,stay_years,martial_status,cate_1,cate_2,cate_3,price,label
0,0,672,0,0,10,0,2,0,2,0,0,8370,1
467664,0,2098,0,0,10,0,2,0,7,0,0,7953,1
467663,0,570,0,0,10,0,2,0,2,3,9,11051,1
428894,0,722,0,0,10,0,2,0,2,3,9,10572,1
428893,0,249,0,0,10,0,2,0,0,1,6,15416,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
544172,5890,2207,1,2,6,1,2,0,5,0,0,12570,0
544173,5890,2843,1,2,6,1,2,0,7,0,0,7799,0
544174,5890,1722,1,2,6,1,2,0,1,3,6,6974,0
544175,5890,1910,1,2,6,1,2,0,0,5,5,19391,0


In [36]:
dataset = 'black-fri'
feat_dict = get_feat_dict(dataset)
feat_dict

{'user_id': 5891,
 'product_id': 3631,
 'gender': 2,
 'age': 7,
 'occupation': 21,
 'city': 3,
 'stay_years': 5,
 'martial_status': 2,
 'cate_1': 20,
 'cate_2': 18,
 'cate_3': 16,
 'price': 23962}

In [30]:
train_feats = ['user_id','product_id','gender','age','occupation','city','stay_years','martial_status','cate_1','cate_2','cate_3','price']

fixlen_feature_columns = [SparseFeat(feat, feat_dict[feat], embedding_dim=4)
								  for feat in train_feats]

linear_feature_columns = fixlen_feature_columns
dnn_feature_columns = fixlen_feature_columns

feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

In [31]:
train_data = sklearn.utils.shuffle(train_data)

train_model_input = {name: train_data[name].values for name in train_feats}
label = train_data['label'].values

In [32]:
model = DeepFM(linear_feature_columns, dnn_feature_columns, task='binary')

model.compile(optimizer=tf.keras.optimizers.Adam(), loss='binary_crossentropy',
              metrics=['AUC', 'Precision', 'Recall'])
model.summary()

Model: "functional_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
user_id (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
product_id (InputLayer)         [(None, 1)]          0                                            
__________________________________________________________________________________________________
gender (InputLayer)             [(None, 1)]          0                                            
__________________________________________________________________________________________________
age (InputLayer)                [(None, 1)]          0                                            
_______________________________________________________________________________________

In [33]:
batch_size = 256
epochs = 20

check_path = saved_model_path + 'deepfm-black-fri-liuyi.ckpt'
checkpoint = tf.keras.callbacks.ModelCheckpoint(check_path, save_weights_only=True,
                                                 save_best_only=True,
                                                 monitor='val_loss',
                                                 )

In [34]:
print('begain training')
model.fit(train_model_input, label,
						batch_size=batch_size, epochs=epochs, verbose=2,
                        callbacks=[checkpoint],
						validation_split=0.2)

begain training
Epoch 1/20
3402/3402 - 30s - loss: 0.5034 - auc: 0.8311 - precision: 0.7330 - recall: 0.7926 - val_loss: 0.4859 - val_auc: 0.8455 - val_precision: 0.7529 - val_recall: 0.7917
Epoch 2/20
3402/3402 - 29s - loss: 0.4662 - auc: 0.8589 - precision: 0.7539 - recall: 0.8282 - val_loss: 0.4678 - val_auc: 0.8592 - val_precision: 0.7626 - val_recall: 0.8140
Epoch 3/20
3402/3402 - 29s - loss: 0.4471 - auc: 0.8728 - precision: 0.7682 - recall: 0.8387 - val_loss: 0.4647 - val_auc: 0.8636 - val_precision: 0.7543 - val_recall: 0.8442
Epoch 4/20
3402/3402 - 29s - loss: 0.4334 - auc: 0.8827 - precision: 0.7776 - recall: 0.8491 - val_loss: 0.4644 - val_auc: 0.8650 - val_precision: 0.7528 - val_recall: 0.8522
Epoch 5/20
3402/3402 - 29s - loss: 0.4236 - auc: 0.8898 - precision: 0.7853 - recall: 0.8541 - val_loss: 0.4692 - val_auc: 0.8641 - val_precision: 0.7658 - val_recall: 0.8261
Epoch 6/20
3402/3402 - 29s - loss: 0.4173 - auc: 0.8943 - precision: 0.7901 - recall: 0.8587 - val_loss: 0.47